
# Context

## Goal of this course

In the previous tutorial we formulated a data problem, chose a dataset and performed some initial steps that helped clean and then explore the structure of our data. In this second course we will start looking at how to prepare our dataset to be compatible with the kind of statistical models that will be able to help us solve the problem we formulated in the previous course.


*Data wrangling* is one of the most important steps in the modelling process. We need to prepare a dataset to be ingested into at least one of many statistical models and make the most of the data we have access to. 

The first step is to make sure we understand the scope of the problem:
- What features of our dataset are we going to use as input to our model (feature engineering)?
- Will our model predict something or will it look for unlabeled structure? (supervised vs unsupervised models)
- If predicting, what will the prediction target be? Is it represented as discrete categories or a linear scale (classification or regression)? 


## Supervised vs Unsupervised learning

The first step here is to acknowledge  whether your data problem is suitable for “supervised” or “unsupervised” learning models. 
 
__Unsupervised learning__ models receive input (_explanatory/predictor/independent_ variables or _features_) but no output (or _dependent/target_ variable)  and, essentially, are a way of discovering latent structure in a set of data (clustering is an example of unsupervised learning). Unsupervised models are very useful when working with unlabelled datasets. These models can then be (and often are) combined with supervised models.
 
__Supervised learning__ models, essentially,  learn a mathematical function between an input (_explanatory/predictor/independent_ variables) and an output (the _dependent/target_ variable). These models are used in situations where you know what you want to predict and have explicit input-output pairs for your model to be trained upon.

In our current project we want to forecast the minutes watched on iPlayer (output or target variable) based on past behavior (input or _explanatory/predictor/independent_ variables). We have input-output pairs and are, therefore, in the supervised learning framework.


## Classification vs Regression

Within the supervised learning class of problems there is a further split, based upon the nature of the target variable: discrete or linear. If a dataset is being used to predict one of a finite set of categories, this is called a classification problem and is tackled by a specific set of models called a classifiers. A simple example of a classification problem is a spam email filter where the text of the email must be classified as either legitimate or not. A further example, now of a multi-classification problem is for handwriting recognition, where a hand-written letter is classified as one of a set of 36 (26 letters and 10 digits).

Alternatively, some data problems require the prediction of linear or non-categorical variables. This class of problems is approached using a class of model called regressors. An example of a regression problem would be to predict the number of hits an article receives by analysing the contents of its headline.

In our current project we want to predict _whether or not_ a user watched any content within two week period (a classification problem) and we also want to know _how much_ content a user watched within a two-week period. We will therefore be using both classifierds and regressors in this project.


# Data wrangling

## Scope

When computing the distribution of our observations among the `twoweek` variable (course 1 - _Exploratory data analysis_) we saw that we had roughly the same amount of data, except for week 0. Here, as it is not clear why it should have less data (meaning it may be a sign of something not being right), we decide to remove this group from all our subsequent analyses.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Download our output dataset from course 1
data = pd.read_csv('iplayer_data_c1.csv')

# Check that it all looks how we expect it to.
data.head()

,user_id,program_id,series_id,genre,programme_duration,streaming_id,start_date_time,time_viewed,weekday,time_of_day,programme_duration_mins,twoweek,min_watched,enriched_genre,hour,enriched_genre_hour,enriched_duration_mins,percentage_watched
0,cd2006,f6d3d8,a282ca,Factual,00:00:21,1486911129420_1,2017-02-12 14:51:24.544,20920.0,weekday_6,Afternoon,0.350000,3,0.348667,Factual,14,Factual,0.350000,0.996190
1,cd2006,b8fbf2,e0480e,Comedy,00:01:51,1484864257965_1,2017-01-19 22:17:04.648,111285.0,weekday_3,Evening,1.850000,1,1.854750,Comedy,22,Drama,1.850000,1.000000
2,cd2006,e2f113,933a1b,Factual,00:00:30,1487099603980_1,2017-02-14 19:12:36.667,29945.0,weekday_1,Evening,0.500000,3,0.499083,Factual,19,Factual,0.500000,0.998167
3,cd2006,0e0916,b68e79,Entertainment,00:01:22,1484773546557_1,2017-01-18 21:05:11.466,82620.0,weekday_2,Evening,1.366667,1,1.377000,Entertainment,21,Drama,1.366667,1.000000
4,cd2006,ca03b9,5d0813,Sport,00:01:37,1486911176609_1,2017-02-12 14:52:08.965,97444.0,weekday_6,Afternoon,1.616667,3,1.624067,Sport,14,Factual,1.616667,1.000000


In [3]:
# Based on the plots in course 1, we will drop week 0
data=data[data['twoweek']>0]

In [4]:
data.twoweek.value_counts().sort_index()

1    67222
2    62112
3    60431
4    51941
5    55668
6    49941
7    51286
8    53410
Name: twoweek, dtype: int64

## Target variables
### Granularity
In our specific problem, we want to forecast what _individual_ users will do. We, therefore, need to pivot our datasets from an events view to a __user view__ to build both our target and features.

We also want to forecast _the next two weeks_ behaviour so we have to aggregate our target metric for the last two weeks of data available, i.e. `twoweek` 8. 

Which metric to build the target on?

### Regression and classification
We will  build two different kind of models:
- one in which our target is the minutes watched in the next two weeks - for this first kind of modeling we are in the __regression__ framework
- another one in which our target is a dummy variable (0/1) and we will forecast whether a user will watch iPlayer or not (total minutes watches > 0) within the next two weeks - here we are in the __classification__ framework


#### Regression

In [5]:
# We need to compute the total minutes watched within twoweek 8 for each user
target_reg=pd.pivot_table(data,
                          values='min_watched',
                          index=['user_id'],
                          columns=['twoweek'],
                          aggfunc=sum)[8].reset_index()

# We fill the NAs with 0: corresponds to the users without observations for the last twoweek
# i.e. who didn't watch anything
target_reg=target_reg.fillna(0)
target_reg.head()

,user_id,8
0,0001c6,0.144833
1,000c1a,318.047633
2,001c53,1.980350
3,001d44,10.059067
4,002b2e,0.000000


#### Classification

In [6]:
# We build the dummy variable based on the minutes watched on this twoweek
target_class=target_reg.copy()
target_class[8]=np.where(target_class[8]>0,1,0)
target_class.head()

,user_id,8
0,0001c6,1
1,000c1a,1
2,001c53,1
3,001d44,1
4,002b2e,0


## Explanatory variables
### User granularity and feature engineering
Here again we need to have a `user_id` granularity. Dealing with datetime data, we could also have considered a `user_id x twoweek` granularity and work with time series. We decided to not work on this framework here - which can be better to get dynamics though - and will then compute a features dataset with one row per user.

As mentioned in the first course, in most situations the process of feature engineering is an iterative one until you get the feature set that neither “underfits” or “overfits” the data. A feature set that does not contain sufficient information regarding the output variable will often result in the model underfitting (this can usually be identified by a high training error). The solution here is often to add more features. If the feature set contains features that are sensitive to  spurious and random elements of the dataset (and not the underlying population it should be an approximation of), overfitting occurs. Overfitting is characterised by low training error and high test error. Overfitting can be tackled by reducing the complexity of your model (often removing features) or use regularisation techniques (https://www.quora.com/What-is-regularization-in-machine-learning). A larger and more diverse training set also helps to reduce overfitting. 

There are various feature selection tools that can be used together with cross-validation to optimise your feature set (e.g. stepwise regression - https://en.wikipedia.org/wiki/Stepwise_regression).  
 
In our project we choose a selection of features that describe how the type viewing habits of a particular user (e.g. “average completion”, “most watched genre”, ”time watched“).

In [7]:
# Create a function that pivots the data based on customer
# and gives us all the features we need
def pivot_data(dataframe):
    #How many minutes did each person watch in each 2 week period
    data=pd.pivot_table(dataframe,values='min_watched', 
                        index=['user_id'],columns=['twoweek'], aggfunc=sum)
    # Fill the weeks they didn't watch in with 0s
    data.fillna(0,inplace=True)
    # How much of average did each viewer watch?
    data['average_completion']=dataframe.groupby('user_id')['percentage_watched'].mean()
    # How many sessions did the person have with us
    data['total_sessions']=dataframe.groupby('user_id')['streaming_id'].nunique()
    # How much did the viewer watch in total this year so far
    data['total_watched']=dataframe.groupby('user_id')['min_watched'].sum()
    # How many times has the viewer watched something
    data['number_watched']=dataframe.groupby('user_id')['streaming_id'].count()
    # Genre most watched by the viewer
    data['most_genre']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                      columns=['enriched_genre'], aggfunc=sum).idxmax(axis=1)
    # Number of genres watched
    data['num_genre']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                     columns=['enriched_genre'], aggfunc=sum).count(axis=1)
    # Favourite day of the week to watch
    data['most_weekday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                        columns=['weekday'], aggfunc=sum).idxmax(axis=1)
    # Number of weekdays watched
    data['num_weekday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                       columns=['weekday'], aggfunc=sum).count(axis=1)
    # Favorite time of day to watch
    data['most_timeday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                        columns=['time_of_day'], aggfunc=sum).idxmax(axis=1)
    # Number of times of day
    data['num_timeday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                       columns=['time_of_day'], aggfunc=sum).count(axis=1)
    return data

In [8]:
# We need to consider here only the "past data", i.e. get rid of the last twoweek observation
# which corresponds to our target
features=pivot_data(data[data['twoweek']<8])
features.reset_index().head()

twoweek,user_id,1,2,3,4,5,6,7,average_completion,total_sessions,total_watched,number_watched,most_genre,num_genre,most_weekday,num_weekday,most_timeday,num_timeday
0,0001c6,16.679200,0.000000,0.0000,0.000000,0.000000,0.152550,0.000000,0.371496,2,16.831750,3,News,1,weekday_1,2,Evening,2
1,000c1a,0.162867,0.147467,107.0984,145.686233,2.286283,100.487767,132.432083,0.233136,28,488.301100,38,Factual,5,weekday_3,6,Morning,3
2,001c53,1.866300,0.000000,0.0000,0.000000,1.309867,0.000000,0.000000,0.489419,3,3.176167,3,News,2,weekday_2,2,Morning,2
3,001d44,0.000000,0.000000,0.0000,14.547700,0.000000,0.000000,0.248017,0.058203,2,14.795717,3,Sport,2,weekday_6,1,Morning,2
4,002b2e,291.477033,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.228233,17,291.477033,21,Factual,5,weekday_2,5,Evening,3


So for each user we have:
- the minutes watched on a 2 weeks basis for the past 14 weeks `1`, `2`,..., `7`

And aggregated on this 14 weeks timeframe:
- the total minutes watched `total_watched`
- the average completion when watching a piece of content `average_completion`
- the number of sessions `total_sessions`
- the number of time a user watched something `number_watched`
- the main genre watched - in terms of minutes and not number of pieces of content - `most_genre`
- the number of different genre watched `num_genre`
- the favourite day of the week to watch - again in minutes watched - `most_weekday`
- the number of differents days of the week a user watched something - `num_weekday`
- the favourite time of the day to watch - again in minutes watched - `most_timeday`
- the number of differents times of the day a user watched something - `num_timeday`

This set of variables constitute our features space to build the models on. Note that we could have imagined lots of other features.

### Dummification
Most models only take in quantitative data. We therefore need to __dummify__ the categorical fields, i.e. we will split the variable in _n_ - the number of different values - dummy (0/1) ones.  

In [9]:
# Turn our categorical variables into bins so that we can run models on this
features=pd.get_dummies(features).reset_index()
features.head()

,user_id,1,2,3,4,5,6,7,average_completion,total_sessions,...,most_weekday_weekday_1,most_weekday_weekday_2,most_weekday_weekday_3,most_weekday_weekday_4,most_weekday_weekday_5,most_weekday_weekday_6,most_timeday_Afternoon,most_timeday_Evening,most_timeday_Morning,most_timeday_Night
0,0001c6,16.679200,0.000000,0.0000,0.000000,0.000000,0.152550,0.000000,0.371496,2,...,1,0,0,0,0,0,0,1,0,0
1,000c1a,0.162867,0.147467,107.0984,145.686233,2.286283,100.487767,132.432083,0.233136,28,...,0,0,1,0,0,0,0,0,1,0
2,001c53,1.866300,0.000000,0.0000,0.000000,1.309867,0.000000,0.000000,0.489419,3,...,0,1,0,0,0,0,0,0,1,0
3,001d44,0.000000,0.000000,0.0000,14.547700,0.000000,0.000000,0.248017,0.058203,2,...,0,0,0,0,0,1,0,0,1,0
4,002b2e,291.477033,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.228233,17,...,0,1,0,0,0,0,0,1,0,0


### Naming

We also change the name of the 2 weeks minutes watched variables to make them generic like `tw_lag7_watched`, `tw_lag5_watched`, ..., `tw_lag1_watched`. If we want to use our model afterwards for forecasting purposes on a new timeframe we don't want to be name-dependent. 

In [10]:
features = features.rename(columns={1:'tw_lag7_watched',
                                    2:'tw_lag6_watched',
                                    3:'tw_lag5_watched',
                                    4:'tw_lag4_watched',
                                    5:'tw_lag3_watched',
                                    6:'tw_lag2_watched',
                                    7:'tw_lag1_watched'})
features.head()

,user_id,tw_lag7_watched,tw_lag6_watched,tw_lag5_watched,tw_lag4_watched,tw_lag3_watched,tw_lag2_watched,tw_lag1_watched,average_completion,total_sessions,...,most_weekday_weekday_1,most_weekday_weekday_2,most_weekday_weekday_3,most_weekday_weekday_4,most_weekday_weekday_5,most_weekday_weekday_6,most_timeday_Afternoon,most_timeday_Evening,most_timeday_Morning,most_timeday_Night
0,0001c6,16.679200,0.000000,0.0000,0.000000,0.000000,0.152550,0.000000,0.371496,2,...,1,0,0,0,0,0,0,1,0,0
1,000c1a,0.162867,0.147467,107.0984,145.686233,2.286283,100.487767,132.432083,0.233136,28,...,0,0,1,0,0,0,0,0,1,0
2,001c53,1.866300,0.000000,0.0000,0.000000,1.309867,0.000000,0.000000,0.489419,3,...,0,1,0,0,0,0,0,0,1,0
3,001d44,0.000000,0.000000,0.0000,14.547700,0.000000,0.000000,0.248017,0.058203,2,...,0,0,0,0,0,1,0,0,1,0
4,002b2e,291.477033,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.228233,17,...,0,1,0,0,0,0,0,1,0,0


## Make sure to have couples of features-target
The last thing we need to do is to make sure that we have pairs of features X and target variable Y for each user. If features or target values are missing for a given user we will get rid of this observation.

We also need to make sure that these observations are in the right order.

In [11]:
# Find the unique users in both the features and the target
users_target=target_reg['user_id'].unique()
users_features=features['user_id'].unique()

# Find those users that are in the target but not in the feature
target_not_feature=[]
for user in users_target:
    if user not in users_features:
        target_not_feature.append(user)

# Find those users that are in the feature but not in the target
feature_not_target=[]
for user in users_features:
    if user not in users_target:
        feature_not_target.append(user)

# Print the size of the two sets
print('In target but not feature:',len(target_not_feature),
      '- In feature but not target:' ,len(feature_not_target))

In target but not feature: 56 - In feature but not target: 0


Remark: it's actually normal to have no one missing in the second case because we built the target variables based on the entire population of the training data. It's more a sanity check here.

We thus need to remove some users who don't have any past behaviour before `twoweek` 8. We actually could set all of their explanatory variables to 0 or default but there are probably new users and the models won't perform well for such profile. Speaking of which, the seniority of the user could be a great feature to consider but need some "business" rules to avoid what we call "left-censoring" issues. 

In [12]:
# We will set the index to the user_id as this will make it easier to drop rows
# Then we drop the rows and then turn the remaining column into an array
target_reg=target_reg.set_index(['user_id'])
target_reg.drop(target_not_feature,inplace=True)
target_reg.reset_index(inplace=True)
target_reg=target_reg[8].values

# Same for the classification
target_class=target_class.set_index(['user_id'])
target_class.drop(target_not_feature,inplace=True)
target_class.reset_index(inplace=True)
target_class=target_class[8].values
        
# Check to make sure the outcome makes sense
print(target_reg[:10])
print(target_class[:10])

[  1.44833333e-01   3.18047633e+02   1.98035000e+00   1.00590667e+01
   0.00000000e+00   4.79261667e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
[1 1 1 1 0 1 0 0 0 0]


In [14]:
# Let's check the size of our datasets
print('Number of samples in the training feature set:',len(features))
print('Number of samples in the training target set (classification):',
      len(target_class))
print('Number of samples in the training target set (regression):',
      len(target_reg))

Number of samples in the training feature set: 9107
Number of samples in the training target set (classification): 9107
Number of samples in the training target set (regression): 9107


## Missing values

The last thing we need to do is to remove any possible missing values. And to get rid of the `user_id` field as it's not a feature for the modeling part. 

Note that we need to keep it somewhere to easily find back our users when doing the forecastings and put our insights into actions. That's why we are using it as our database index.

In [13]:
# We will fill remaining missing values with 0s as we don't know any better
features=features.set_index(['user_id'])
features.fillna(0,inplace=True)
features.head()

,tw_lag7_watched,tw_lag6_watched,tw_lag5_watched,tw_lag4_watched,tw_lag3_watched,tw_lag2_watched,tw_lag1_watched,average_completion,total_sessions,total_watched,...,most_weekday_weekday_1,most_weekday_weekday_2,most_weekday_weekday_3,most_weekday_weekday_4,most_weekday_weekday_5,most_weekday_weekday_6,most_timeday_Afternoon,most_timeday_Evening,most_timeday_Morning,most_timeday_Night
user_id,,,,,,,,,,,,,,,,,,,,,
0001c6,16.679200,0.000000,0.0000,0.000000,0.000000,0.152550,0.000000,0.371496,2,16.831750,...,1,0,0,0,0,0,0,1,0,0
000c1a,0.162867,0.147467,107.0984,145.686233,2.286283,100.487767,132.432083,0.233136,28,488.301100,...,0,0,1,0,0,0,0,0,1,0
001c53,1.866300,0.000000,0.0000,0.000000,1.309867,0.000000,0.000000,0.489419,3,3.176167,...,0,1,0,0,0,0,0,0,1,0
001d44,0.000000,0.000000,0.0000,14.547700,0.000000,0.000000,0.248017,0.058203,2,14.795717,...,0,0,0,0,0,1,0,0,1,0
002b2e,291.477033,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.228233,17,291.477033,...,0,1,0,0,0,0,0,1,0,0


__TO DO:__
- COMMENTS
- xxx

__Save new datasets - or create a class to call for the other notebook ?__

In [16]:
features.to_csv('features.csv')
np.savetxt('target.txt',(target_reg,target_class))